# СРС №5. Обслуживание простых деревьев. Таблица замыканий
## Задание.
1. Реализовать структуру таблиц, реализовывающую структуру для хранения деревьев «Таблица замыканий» и позволяющую хранить данные выбранной тематики. Тематика может быть выбрана из списка предложенных (см. Предлагаемые темы) или предложена студентом.
2. Наполнить данными реализованную структуру. Наполнение и генерация данных должны производиться с программным кодом. Таблица с деревом должна содержать не менее 150 строк, каждый узел, не являющийся листом должен иметь от 2 до 5 потомков, общая глубина дерева должна быть не менее 10.
3. Реализовать следующие операции в виде функций программной бибилотеки с текстовым или графическим интерфейсом:
- добавление листа;
- удаление листа;
- удаление поддерева;
- перемещение листа;
- перемещение поддерева;
- удаление узла без поддерева;
- перемещение узла без поддерева;
- получение прямых потомков;
- получение прямого родителя;
- получение всех потомков (с сохранением иерархичности);
- получение всех родителей (с сохранением иерархичности);

In [1]:
import mysql.connector
import pandas as pd
import random 
from faker import Faker
from mysql.connector import connect, Error
from queue import Queue
fake = Faker('ru_RU')
r_host = "localhost"
r_user = "root"
r_password = "aryuna"
r_db = "closure_table"
connection = connect(host=r_host, user=r_user, password=r_password,); db_cur = connection.cursor()
db_cur.execute(f"CREATE DATABASE IF NOT EXISTS {r_db}")
conn = connect(
    host=r_host, 
    user=r_user,
    password=r_password,
    database = r_db,
)
cur = conn.cursor()
# tree хранит данные о том, что путь вообще существует 

In [2]:
def AddChildren(ancestors):
    generation = []
    while not ancestors.empty():
        descend = random.randint(2, 5) #сколько детей будет у текущего
        new_parent = ancestors.get()
        for i in range(descend): #по очереди создаём
            #print(f"{i+1} по счёту ребенок, {new_parent} - родитель. Детей должно быть {descend}")
            cur.execute(f'INSERT INTO brands (brand) VALUES ("{fake.company()}")')
            cur.execute(f'''INSERT INTO tree (ancestor, descendant, depth)
                SELECT ancestor, LAST_INSERT_ID(), depth + 1 FROM tree WHERE descendant = {new_parent}
                        UNION ALL 
                SELECT LAST_INSERT_ID(), LAST_INSERT_ID(), 0''')
#1-й добавляет данные о сущ-ии, формирует пути от предков родителя до вставленного узла.
#2-й вставляет путь от узла до самого себя.
            conn.commit()
        cur.execute(f"SELECT descendant FROM tree WHERE ancestor = {new_parent} AND descendant <> ancestor")
        generation.append(cur.fetchall())
    this_gen_par = []
    for _list in generation: #списки развернуть и сделать не список списков, а просто список
        for x in _list:
            this_gen_par.append(int(x[0])) # x[0] чтоб избавиться от кортежа
    #print(f'\t {this_gen_par}')
    return this_gen_par
def NewParents(ancestors, gen):
#сначала достаём всё нынешнее поколение
#потом генерим кол-во детей, которые родят
#а потом рандомсэмплим тех, кто родит и всех их кладём в ancestors
    chil_num = random.randint(1, len(gen)) #сначала генерим то, сколько из детей станет родителями
    gen = random.sample(gen, chil_num) #получаем тех, кто станет родителями
    for x in gen:
        ancestors.put(x) #кладём в очередь
def TreeGenerator(d):
    ancestors = Queue() #очередь родителей
    #создание корня, id будет 1 из-за автоинкремента
    cur.execute(f'INSERT INTO brands (brand) VALUES ("{fake.company()}")')
    cur.execute(f"INSERT INTO tree (ancestor, descendant) VALUES (LAST_INSERT_ID(), LAST_INSERT_ID())")
    conn.commit()
    ancestors.put(1)
    new_generation = AddChildren(ancestors) #получение нынешнего поколения
    #print(ancestors.qsize()) #0 после 1-го раза - потому что в процессе добавления детей всё удалилось
    depth = 1 #Т.к. уже случился AddChildren 
    while (depth < d):
        NewParents(ancestors, new_generation)
        new_generation = AddChildren(ancestors)
        depth += 1

In [3]:
def AddLeaf(title, parent):
    cur.execute(f"INSERT INTO brands (brand) VALUES ('{title}')")
    cur.execute(f'''INSERT INTO tree (ancestor, descendant, depth)
SELECT ancestor, LAST_INSERT_ID(), depth + 1 FROM tree WHERE descendant = {parent}
UNION ALL 
SELECT LAST_INSERT_ID(), LAST_INSERT_ID(), 0''')
    # вставка ссылок на родителя в потомках с глубиной +1 и само дитя непосредственно
    conn.commit()
def DelSubTreeOrLeaf(v): 
    cur.execute(f"""WITH willdelete AS (SELECT descendant FROM tree WHERE ancestor = {v})
DELETE FROM tree WHERE descendant IN (SELECT * FROM willdelete)""")
    conn.commit()
def MoveSubTreeOrLeaf(v, change):
    cur.execute(f"""WITH children AS (SELECT descendant FROM tree WHERE ancestor = {v}),
        parents AS (SELECT ancestor FROM tree WHERE descendant = {v} AND ancestor <> descendant)
    DELETE FROM tree WHERE descendant IN (SELECT * FROM children) AND ancestor IN (SELECT * FROM parents) """)
    conn.commit() #удаление ссылок на поддерево в предках, кроме себя
    cur.execute(f"""INSERT INTO tree (ancestor, descendant, depth)
SELECT supertree.ancestor, subtree.descendant, supertree.depth + subtree.depth + 1 
FROM tree AS supertree CROSS JOIN tree AS subtree
WHERE supertree.descendant = {change} AND subtree.ancestor = {v}
""") #UNION ALL SELECT {v}, {v}, 0
    conn.commit() #добавили детей
def DelNodeWithoutTree(node):
    cur.execute(f"""WITH children AS (SELECT descendant FROM tree WHERE ancestor = {node})
UPDATE tree SET depth = depth - 1 WHERE descendant IN (SELECT * FROM children) AND ancestor <> descendant""")
    cur.execute(f"""DELETE FROM tree WHERE descendant = {node} OR ancestor = {node}""")
    conn.commit()
def MoveNodeWithoutTree(node, change):
    cur.execute(f"""WITH children AS (SELECT descendant FROM tree WHERE ancestor = {node})
UPDATE tree SET depth = depth - 1 WHERE descendant IN (SELECT * FROM children) AND ancestor <> descendant""") 
    conn.commit() #перенос детей
    cur.execute(f"DELETE FROM tree WHERE ancestor = {node} OR descendant = {node}")
    cur.execute(f'''INSERT INTO tree (ancestor, descendant, depth)
SELECT ancestor, {node}, depth + 1 FROM tree WHERE descendant = {change}
UNION ALL 
SELECT {node}, {node}, 0''') #переместили
    conn.commit()
def GetAllParents(v): 
    cur.execute(f"SELECT id, brand, depth FROM tree JOIN brands ON ancestor = id AND descendant = {v} ORDER BY depth DESC")
    for i in cur:
        print(i)
def GetAllChildren(v):
    cur.execute(f"SELECT depth, id, brand FROM tree JOIN brands ON descendant = id AND ancestor = {v} ORDER BY depth")
    for i in cur:
        print(i)
def GetParent(v):
    cur.execute(f"SELECT id, brand, depth FROM tree JOIN brands ON ancestor = id AND descendant = {v} AND depth = 1")
    return cur.fetchone()[0]
def GetChildren(v):
    cur.execute(f"SELECT id, brand, depth FROM tree JOIN brands ON descendant = id AND ancestor = {v} AND depth = 1")
    children = [x[0] for x in cur.fetchall()]
    return children

In [4]:
def Checking():
    cur.execute("SELECT MAX(depth) FROM tree")
    d = cur.fetchone()[0]
    cur.execute("SELECT count(id) FROM brands")
    n = cur.fetchone()[0]
    print(f"Количество элементов - {n}")
    print(f"Глубина дерева - {d}")
    cur.execute(f"""SELECT id, brand, COUNT(descendant) FROM tree JOIN brands ON ancestor = id AND depth = 1 GROUP BY id
HAVING COUNT(descendant) NOT BETWEEN 2 AND 5""")
    # путь - посл-ть РЁБЕР, в которой каждые 2 соседних р содержат ОБЩУЮ ВЕРШИНУ, depth - длина пути - кол-во РЁБЕР
    # путь в дереве - последовательность вершин, вершины соединены рёбрами
    check = cur.fetchone()
    print("\t Всё ок, детей от 2 до 5 у всех. " if check is None else "\t ЧТО-ТО ПОШЛО У КАКОЙ-ТО ВЕРШИНЫ НЕ ТАК")
def Interface():
    tab = ''
    a = input("Введите вершину для махинаций. Если понадобится что-то ещё, введёте потом. ")
    while tab != 'Всё':
        cur = conn.cursor()
        cur.execute(f"SELECT ancestor FROM tree where ancestor = {a} and descendant = {a}")
        check = cur.fetchone()
        if check is None:
            a = input("Этой вершины нет, давай по новой: ")
            continue
        tab = input("Введите число от 1 до 9. Или 0: ")
        if tab == '1':
            print(f"Родитель вершины {a}: {GetParent(a)}")
        elif tab == '2':
            print(f"Прямые потомки вершины {a}: {GetChildren(a)}")
        elif tab == '3':
            name = input(f"В вершину {a} добавляется лист. Введите его имя: ")
            AddLeaf(name, a)
        elif tab == '4':
            print(f'Удаляется вершина {a} (с поддеревом, если оно есть). ')
            DelSubTreeOrLeaf(a)
        elif tab == '5':
            new = input(f"Куда переместить вершину {a} (с поддеревом, если оно есть)? ")
            MoveSubTreeOrLeaf(a, new)
        elif tab == '6':
            print(f'Удаляется вершина {a} (без поддерева). ')
            DelNodeWithoutTree(a)
        elif tab == '7':
            new = input(f"Куда переместить вершину {a} (без поддерева)? ")
            MoveNodeWithoutTree(a, new)
        elif tab == '8':
            print(f"Получение ВСЕХ ПОТОМКОВ {a}!!! ")
            GetAllChildren(a)
        elif tab == '9':
            print(f"Получение ВСЕХ РОДИТЕЛЕЙ до {a}!!! ")
            GetAllParents(a)
        elif tab == '0':
            a = input("Новая вершина для махинаций: ")
        elif tab == 'Проверка':
            Checking()
        elif tab == "Всё" or tab == ".":
            print("Oк. ")
        else:
            print("Вы ввели что-то не то... ")
    print("До свидания.")

# ы
    1 - прямой родитель
    2 - прямые потомки
    3 - добавление листа
    4 - удаление листа/поддерева
    5 - перемещение листа/поддерева
    6 - удаление узла без поддерева
    7 - перемещение узла без поддерева
    8 - получение всех потомков (с сохранением иерархичности)
    9 - получение всех родителей (с сохранением иерархичности)
    0 - выбор другой вершины

drop = ("DROP TABLE IF EXISTS brands, tree")
brands = ("""CREATE TABLE IF NOT EXISTS brands(
     id INT PRIMARY KEY AUTO_INCREMENT,
     brand VARCHAR(100)
    )
    """)
tree = ("""CREATE TABLE IF NOT EXISTS tree(
     ancestor INT NOT NULL, 
     descendant INT NOT NULL, 
     depth INT NOT NULL DEFAULT 0,
     PRIMARY KEY (ancestor, descendant), 
     FOREIGN KEY (ancestor) REFERENCES brands(id), 
     FOREIGN KEY (descendant) REFERENCES brands(id)
    )""")
cur.execute(drop)
cur.execute(brands)
cur.execute(tree)
conn.commit()
TreeGenerator(10)

In [5]:
Interface()

Введите вершину для махинаций. Если понадобится что-то ещё, введёте потом. 99
Введите число от 1 до 9. Или 0: Проверка
Количество элементов - 1494
Глубина дерева - 10
	 Всё ок, детей от 2 до 5 у всех. 
Введите число от 1 до 9. Или 0: Всё
Oк. 
До свидания.


In [ ]:
cur.execute("ALTER TABLE tree ADD depth INT NOT NULL DEFAULT 0")
conn.commit()